<a href="https://colab.research.google.com/github/minsaee/ai_chat_python/blob/master/901_TTS_STT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


TTS(Text to Speech)    
STT(Speech To Text)      



<<휴대폰 마이크 이용하기>>  
휴대폰에서 -> 설정 -> 휴대전화정보 (7번 클릭) -> 개발자 옵션이 보이면 클릭-> USB 디버깅 활성화

https://reincubate.com/ko/camo/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ai_chat_python

Mounted at /content/drive
/content/drive/MyDrive/ai_chat_python


In [2]:
!pip install gTTS

### 영어문장

In [8]:
from gtts import gTTS

text = 'Can I help you'
file_name = 'sample.mp3'
tts_en = gTTS(text=text, lang='en')
tts_en.save(file_name)


#https://luvris2.tistory.com/123
from IPython.display import Audio
Audio(file_name,autoplay=True)


### 한글 문장

In [9]:
from gtts import gTTS

text = '안녕하세요. 반갑습니다. korea hello'
file_name = 'sample.mp3'
tts_en = gTTS(text=text, lang='ko')
tts_en.save(file_name)


#https://luvris2.tistory.com/123
from IPython.display import Audio
Audio(file_name,autoplay=True)

In [ ]:
from gtts import gTTS

with open('./source/sample.txt') as fp:
  text = fp.read()

file_name = 'sample.mp3'
tts_en = gTTS(text=text, lang='ko')
tts_en.save(file_name)


#https://luvris2.tistory.com/123
from IPython.display import Audio
Audio(file_name,autoplay=True)

## STT(Speech To Text)

In [4]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 44.3 MB/s eta 0:00:00


In [5]:
# !pip install PyAudio 설치할때 아래와 같은 오류가  발생한다.
#Could not build wheels for PyAudio, which is required to install pyproject.toml-based projects
#https://stackoverflow.com/questions/66412902/cannot-install-pyaudio-in-google-colab

!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (320 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 120903 files and directories currently installed.)
Pre

In [6]:
!pip install ffmpeg-python

In [7]:
!pip install PyAudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 809.1 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyAudio: filename=PyAudio-0.2.14-cp310-cp310-linux_x86_64.whl size=63860 sha256=97712616523f1a11d55c216c999c91b75b625c069971754a153a73bc80d6301e
  Stored in directory: /root/.cache/pip/wheels/d6/21/f4/0b51d41ba79e51b16295cbb096ec49f334792814d545b508c5
Successfully built PyAudio


In [ ]:
try:
  import io
  import ffmpeg
  import numpy as np

  # Only available in Google Colab
  from google.colab.output import eval_js

  from IPython.display import HTML, Audio
  from scipy.io.wavfile import write, read as wav_read
  from base64 import b64decode
  from os.path import isfile

  AUDIO_HTML = """
  <script>
  var my_div = document.createElement("DIV");
  var my_p = document.createElement("P");
  var my_btn = document.createElement("BUTTON");
  var t = document.createTextNode("Starting recording...");

  my_btn.appendChild(t);
  my_div.appendChild(my_btn);
  document.body.appendChild(my_div);

  var base64data = 0;
  var reader;
  var recorder, gumStream;
  var recordButton = my_btn;

  var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
      bitsPerSecond: 16000,
      mimeType : 'audio/webm;codecs=opus' //codecs=pcm
    };
    recorder = new MediaRecorder(stream, options);
    //recorder = new MediaRecorder(stream);

    recorder.ondataavailable = function(e) {
      var url = URL.createObjectURL(e.data);
      var preview = document.createElement('audio');
      preview.controls = true;
      preview.src = url;
      document.body.appendChild(preview);

      reader = new FileReader();
      reader.readAsDataURL(e.data);
      reader.onloadend = function() {
        base64data = reader.result;
        //console.log("reader.onloadend: " + base64data);
      }
    };
    recorder.start();
    recordButton.innerText = "🔴 Recording... press to STOP";
  };

  navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

  function toggleRecording() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... please wait!";
    }
  }

  // https://stackoverflow.com/a/951057
  function sleep(ms) {
    return new Promise(resolve => setTimeout(resolve, ms));
  }

  var data = new Promise(resolve => {
    recordButton.onclick = () => {
      toggleRecording();

      sleep(2000).then(() => {
        // wait 2000ms for the data to be available...
        //console.log("resolve data: " + base64data);
        resolve(base64data.toString());
      });
    }
  });

  function doneRecording(recording_file) {
    my_div.removeChild(recordButton);
    my_p.innerText = recording_file;
    my_div.appendChild(my_p);
  }

  </script>
  """

  def get_audio():
    display(HTML(AUDIO_HTML))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])

    process = (ffmpeg
      .input('pipe:0')
      .output('pipe:1', format='wav')
      .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)

    riff_chunk_size = len(output) - 8
    # Break up the chunk size into four bytes, held in b.
    q = riff_chunk_size
    b = []
    for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

    # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    return audio, sr

  recording_file = "recording3.wav" #@param {type:"string"}

  if isfile(recording_file):
    print(f"{recording_file} already exists, if you want to create another recording with the same name, delete it first")
  else:
    # record microphone
    print('마이크 대기')
    audio, sr = get_audio()

    # write recording
    write(recording_file, sr, audio)


    eval_js(f'doneRecording("{recording_file}")')
except ImportError:
  print("Recording only available in Google Colab")

마이크 대기


In [ ]:
import speech_recognition as sr

AUDIO_FILE = 'recording3.wav'
r = sr.Recognizer()

with sr.AudioFile(AUDIO_FILE) as source:
    audio = r.record(source)
text = r.recognize_google(audio, language='ko')
print(text)

안녕하세요 나는 책 보스입니다 반갑습니다 안녕하세요 나는 책 꽃입니다 반갑습니다
